# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
pd.DataFrame()

""


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secretsp import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 97367976, 'calculationPrice': 'iexlasttrade', 'change': 4.14, 'changePercent': 0.02889, 'close': 0, 'closeSource': 'aflfcoii', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 143.13, 'iexCloseTime': 1746274352693, 'iexLastUpdated': 1715005579260, 'iexMarketPercent': 0.01941942900559409, 'iexOpen': 142, 'iexOpenTime': 1683188553876, 'iexRealtimePrice': 143.41, 'iexRealtimeSize': 419, 'iexVolume': 1653979, 'lastTradeTime': 1673808786160, 'latestPrice': 142.49, 'latestSource': 'IEX Last Trade', 'latestTime': 'October 17, 2022', 'latestUpdate': 1747573924354, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2339025240428, 'oddLotDelayedPrice':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2339025240428


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True
)

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\3975751442.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,142.49,2339025240428,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [20]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

...

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\61349748.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Tem

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.540,38986454429,N/A
1,AAL,13.440,8858965255,N/A
2,AAP,177.210,10308879667,N/A
3,AAPL,149.330,2359064935859,N/A
4,ABBV,146.720,266024582327,N/A
...,...,...,...,...
496,YUM,110.370,32681253135,N/A
497,ZBH,110.890,24141005324,N/A
498,ZBRA,268.740,13600090429,N/A
499,ZION,52.642,7861107488,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield  lst[i:i + n]

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
        
final_dataframe

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\3394347993.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yield  lst[i:i + n]
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecate

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppDa

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppDa

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppDa

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\gigio\AppDa

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\2727217901.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.750,38804576305,N/A
1,AAL,13.825,8966525154,N/A
2,AAP,171.600,10259213761,N/A
3,AAPL,142.510,2401842248650,N/A
4,ABBV,151.330,265040239992,N/A
...,...,...,...,...
496,YUM,111.910,32616404221,N/A
497,ZBH,115.700,23563460265,N/A
498,ZBRA,262.280,13944879483,N/A
499,ZION,52.961,7882803201,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [13]:
portifolio_size = input('Enter the value of your portifolio: ')

try:
    val = float(portifolio_size)
except ValueError:
    print("that's not a number, please try again")
    portifolio_size = input('Enter the value of your portifolio: ')
    val = float(portifolio_size)

Enter the value of your portifolio: 10000000


In [14]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

#number_of_apple_shares = position_size / 500
#print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.750,38804576305,150
1,AAL,13.825,8966525154,1443
2,AAP,171.600,10259213761,116
3,AAPL,142.510,2401842248650,140
4,ABBV,151.330,265040239992,131
...,...,...,...,...
496,YUM,111.910,32616404221,178
497,ZBH,115.700,23563460265,172
498,ZBRA,262.280,13944879483,76
499,ZION,52.961,7882803201,376


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [15]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [16]:
background_color = '0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [17]:
#writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
#writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [18]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [19]:
writer.save()

C:\Users\gigio\AppData\Local\Temp\ipykernel_19308\934276808.py:1: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()
